<a href="https://colab.research.google.com/github/yinma-ml/machine-learning/blob/master/Gender_detection_XGBoost_lightGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
## Gender dectection based on human speech

**XGBoost**

In [10]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model.logistic import LogisticRegression
from xgboost import XGBClassifier
#import catboost as cb

In [11]:
data = pd.read_csv('/voice.csv')  #3168 rows × 21 columns

In [12]:
print(data.head())
print(data.isnull().sum())  #no null value

   meanfreq        sd    median       Q25  ...    maxdom   dfrange   modindx  label
0  0.059781  0.064241  0.032027  0.015071  ...  0.007812  0.000000  0.000000   male
1  0.066009  0.067310  0.040229  0.019414  ...  0.054688  0.046875  0.052632   male
2  0.077316  0.083829  0.036718  0.008701  ...  0.015625  0.007812  0.046512   male
3  0.151228  0.072111  0.158011  0.096582  ...  0.562500  0.554688  0.247119   male
4  0.135120  0.079146  0.124656  0.078720  ...  5.484375  5.476562  0.208274   male

[5 rows x 21 columns]
meanfreq    0
sd          0
median      0
Q25         0
Q75         0
IQR         0
skew        0
kurt        0
sp.ent      0
sfm         0
mode        0
centroid    0
meanfun     0
minfun      0
maxfun      0
meandom     0
mindom      0
maxdom      0
dfrange     0
modindx     0
label       0
dtype: int64


In [13]:
print('sample:{}'.format(data.shape[0]))
print('male：{}'.format(data[data.label=='male'].shape[0]))
print('female：{}'.format(data[data.label=='female'].shape[0]))

sample:3168
male：1584
female：1584


In [14]:
X = data.iloc[:,:-1]  #everything except the last column
y = data.iloc[:,-1]
print(X.head)
print(y.head)

<bound method NDFrame.head of       meanfreq        sd    median  ...    maxdom   dfrange   modindx
0     0.059781  0.064241  0.032027  ...  0.007812  0.000000  0.000000
1     0.066009  0.067310  0.040229  ...  0.054688  0.046875  0.052632
2     0.077316  0.083829  0.036718  ...  0.015625  0.007812  0.046512
3     0.151228  0.072111  0.158011  ...  0.562500  0.554688  0.247119
4     0.135120  0.079146  0.124656  ...  5.484375  5.476562  0.208274
...        ...       ...       ...  ...       ...       ...       ...
3163  0.131884  0.084734  0.153707  ...  4.210938  4.203125  0.161929
3164  0.116221  0.089221  0.076758  ...  3.679688  3.640625  0.277897
3165  0.142056  0.095798  0.183731  ...  2.937500  2.929688  0.194759
3166  0.143659  0.090628  0.184976  ...  3.593750  3.585938  0.311002
3167  0.165509  0.092884  0.183044  ...  0.554688  0.546875  0.350000

[3168 rows x 20 columns]>
<bound method NDFrame.head of 0         male
1         male
2         male
3         male
4         mal

In [15]:
# 
lbe = LabelEncoder()
y = lbe.fit_transform(y)
print(y)


[1 1 1 ... 0 0 0]


In [16]:
# normalization
scaler = StandardScaler()
X = scaler.fit_transform(X)
print(X)

[[-4.04924806  0.4273553  -4.22490077 ... -1.43142165 -1.41913712
  -1.45477229]
 [-3.84105325  0.6116695  -3.99929342 ... -1.41810716 -1.4058184
  -1.01410294]
 [-3.46306647  1.60384791 -4.09585052 ... -1.42920257 -1.41691733
  -1.06534356]
 ...
 [-1.29877326  2.32272355 -0.05197279 ... -0.5992661  -0.58671739
   0.17588664]
 [-1.2452018   2.012196   -0.01772849 ... -0.41286326 -0.40025537
   1.14916112]
 [-0.51474626  2.14765111 -0.07087873 ... -1.27608595 -1.2637521
   1.47567886]]


In [17]:
# split train test sets, test size=0.1
X_train, X_test = train_test_split(X, test_size=0.1)
y_train, y_test = train_test_split(y, test_size=0.1)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_seed=0.3)

In [18]:
## XGBoost (from sklearn)

In [19]:
model = XGBClassifier(booster='gbtree',     #Can be gbtree, gblinear or dart; gbtree and dart use tree based models while gblinear uses linear functions.
                      objective='binary:logistic',
                      learning_rate=0.01,   # default=0.3
                      n_estimators=10,      # number of trees-10 trees for xgboost
                      max_depth=4,          # default=6, Maximum depth of a tree. Increasing this value will make the model more complex and more likely to overfit.
                      min_child_weight=1,   # default=1, Minimum sum of instance weight (hessian) needed in a child.The larger min_child_weight is, the more conservative the algorithm will be.
                      gamma=0,              # default=0, Minimum loss reduction required to make a further partition on a leaf node of the tree. The larger gamma is, the more conservative the algorithm will be.
                      subsample=0.8,        # default=1, Subsample ratio of the training instances. Setting it to 0.5 means that XGBoost would randomly sample half of the training data prior to growing trees. and this will prevent overfitting.
                      colsample_bytree=0.8, # default=1, use all samples. subsample ratio of columns when constructing each tree. Subsampling occurs once for every tree constructed.
                      scale_pos_weight=0.9, # default=1, Control the balance of positive and negative weights, useful for unbalanced classes. A typical value to consider: sum(negative instances) / sum(positive instances)
                      reg_alpha=0,          # default=0, L1 regularization term on weights. Increasing this value will make model more conservative.
                      reg_lambda=1,         # default=1, L2 regularization term on weights. Increasing this value will make model more conservative.
                      random_state=27,      #  
                      slient=0
                     )
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0,
              learning_rate=0.01, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=None, n_estimators=10, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=27,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=0.9, seed=None,
              silent=None, slient=0, subsample=0.8, verbosity=1)

In [20]:
y_pred = model.predict(X_test)
print('XGBoost accuracy：%.4lf' % metrics.accuracy_score(y_test, y_pred))

XGBoost accuracy：0.4700


In [21]:
## lightGBM
from lightgbm import LGBMClassifier

In [23]:
model_lgb = LGBMClassifier(boosting_type='gbdt',
                       objective='binary',  #  
                       learning_rate=0.01,  #  
                       n_estimators=10,     #  
                       num_leaves=2**7,     #  
                       min_split_gain=0,    #  
                       subsample=0.8,       #  
                       colsample_bytree=0.8,# 
                       scale_pos_weight=0.9,#  
                       random_state=33,
                       reg_alpha=0,         # L1 
                       reg_lambda=1,        # L2 
                      )
model_lgb.fit(X_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.8,
               importance_type='split', learning_rate=0.01, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0,
               n_estimators=10, n_jobs=-1, num_leaves=128, objective='binary',
               random_state=33, reg_alpha=0, reg_lambda=1, scale_pos_weight=0.9,
               silent=True, subsample=0.8, subsample_for_bin=200000,
               subsample_freq=0)

In [24]:
y_pred = model_lgb.predict(X_test)
y_pred = [1 if x>=0.5 else 0 for x in y_pred]
print("LightGBM prediction：", y_pred)

print("LightGBM accuracy：%.4lf" % metrics.accuracy_score(y_test, y_pred))

LightGBM prediction： [1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1]
LightGBM accuracy：0.5079


In [ ]:
## CatBoost

In [ ]:
'''
model = cb.CatBoostClassifier(iterations=1000,
                             depth=7,
                             learning_rate=0.01,
                             loss_function='Logloss',
                             #eval_metrics='AUC',
                             logging_level='Verbose',
                             metric_period=50)

categorical_features_indices = []
for i in range(len(X_train.columns)):
    if X_train.columns.values[i] in attr:
        categorical_features_indices.append(i)
print(categorical_features_indices)

model.fit(X_train, y_train, eval_set=(X_test y_test), cat_features=categorical_features_indices)
predict = model.predict(X_test)
'''